## Basics of Statistical Analysis


https://www.sfu.ca/~mjbrydon/tutorials/BAinPy/01_intro.html

In [ ]:
import pandas as pd
import seaborn as sns
import os

In [ ]:
os.getcwd()

In [ ]:
data = pd.read_json('C:/Users/user/Desktop/Advanced Data Analysis with Python 9.9.22/Datasets/twitter_data_ML.json')
data.columns

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
data = data[['emoji_n', 'tweet_length', 'tweet_unique_length',
       'province_codes', 'female', 'followers_count', 'following_count',
       'age_group', 'university_rank', 'university', 'n_pos_sent', 'rank_dummy', 
       'big_cities', 'avr_w_length', 'positive']]

I recently produced two variables: ```avr_w_length, positive```

Following command line shows the operation.

You should not run the following command line, since these variables are already defined.

In [ ]:
def average(numbers):
    if len(numbers)>0:
        return sum(numbers)/len(numbers)
    else:
        return 0

avr_w_length = []

tweets = data['tweets'].to_numpy()

for i in range(len(tweets)):
    words = str(tweets[i]).split()
    lengths = [len(word) for word in words]
    avr_w_length.append(average(lengths))
    
data['avr_w_length'] = pd.DataFrame(avr_w_length)

#------------------------------------------------------------------

data['positive'] = pd.cut(data['n_pos_sent'],[0,3,5],labels=[0,1])

In [ ]:
data['female']
#male=0, female=1

### histogram

In [ ]:
data['university_rank'].hist()

In [ ]:
sns.displot(x='university_rank', row='female', data=data, linewidth=0, kde=True);

### overlaying kernel density plots

In [ ]:
sns.kdeplot(x='university_rank', hue='female', data=data, shade=True)

### t-test

In [ ]:
female_rank = data[data['female'] == 1]['university_rank']
male_rank = data[data['female'] == 0]['university_rank']

In [ ]:
female_rank

In [ ]:
from scipy import stats

In [ ]:
#Whether variances are equal or not, null hypothesis is that they are equal
stats.levene(female_rank, male_rank)

In [ ]:
#pip install statsmodels

In [ ]:
import statsmodels.stats.api as sms

In [ ]:
model = sms.CompareMeans.from_data(data[data['female'] == 1]['university_rank'], data[data['female'] == 0]['university_rank'])
model.summary(usevar='pooled') #pooled or unequal, you have two options

The difference between males and females are not statistically significant at 0.05 p_value. However, the interpretation might change if we reconsider the output at 0.1 p_value.

### Cross-tabs

In [ ]:
#age ranges: -18,19-29,30-39,40+

contab_freq = pd.crosstab(
    data['female'],
    data['age_group'],
    margins = False
    #, normalize='index'
   )
contab_freq

In [ ]:
chi = stats.chi2_contingency(contab_freq)
chi

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2_contingency.html

### Correlation and scatterplots

In [ ]:
import seaborn as sns
sns.scatterplot(x="avr_w_length", y="university_rank", data=data)

In [ ]:
ax = sns.scatterplot(x="avr_w_length", y="university_rank", data=data)
ax.set_title("Average word length vs University ranking")
ax.set_xlabel("the average word length used in tweets")

In [ ]:
sns.lmplot(x="avr_w_length", y="university_rank", data=data)

In [ ]:
sns.lmplot(x="avr_w_length", y="university_rank",hue="big_cities", data=data)

In [ ]:
from scipy import stats
stats.pearsonr(data['avr_w_length'], data['university_rank'])
#coefficient correlation and p-value

In [ ]:
data.corr()

In [ ]:
#Seaborn heatmap
sns.heatmap(data.corr())

In [ ]:
sns.heatmap(data.corr(), cmap="YlGnBu", annot=True)

### Simple regression analysis

In [ ]:
data

In [ ]:
import statsmodels.api as sm

In [ ]:
y = data['university_rank']
X = data['avr_w_length']

In [ ]:
X

In [ ]:
X = sm.add_constant(X)
#In OLS models, we have always Beta zero coefficient as an explanatory variable, which is known as the intercept of the line.

In [ ]:
X

In [ ]:
model = sm.OLS(y, X, missing='drop')
model_result = model.fit()
model_result.summary()

### Regression diagnostics

#### Histogram of residuals

In [ ]:
import seaborn as sns
sns.histplot(model_result.resid)

In [ ]:
from scipy import stats
mu, std = stats.norm.fit(model_result.resid)
mu, std

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots()
# plot the residuals
sns.histplot(x=model_result.resid, ax=ax, stat="density", linewidth=0, kde=True)
ax.set(title="Distribution of residuals", xlabel="residual")

# plot corresponding normal curve
xmin, xmax = plt.xlim() # the maximum x values from the histogram above
x = np.linspace(xmin, xmax, 100) # generate some x values
p = stats.norm.pdf(x, mu, std) # calculate the y values for the normal curve
sns.lineplot(x=x, y=p, color="orange", ax=ax)
plt.show()

#### Boxplot of residuals

In [ ]:
sns.boxplot(x=model_result.resid, showmeans=True)

#### Quantile - quantile plot of residuals

In [ ]:
sm.qqplot(model_result.resid, line="s")

In [ ]:
fig = sm.qqplot(model_result.resid, line="s")
plt.show()

#### Fit plot

In [ ]:
fig = sm.graphics.plot_fit(model_result, 1, vlines=False)
plt.show()

In [ ]:
model_result.fittedvalues

In [ ]:
Y_max = y.max()
Y_min = y.min()

ax = sns.scatterplot(x=model_result.fittedvalues, y=y)
ax.set(ylim=(Y_min, Y_max))
ax.set(xlim=(Y_min, Y_max)) #revise Y_min and Y_max with 450 and 550
ax.set_xlabel("Predicted value of Univ. Ranking")
ax.set_ylabel("Observed value of Univ. Ranking")

X_ref = Y_ref = np.linspace(Y_min, Y_max, 100)
plt.plot(X_ref, Y_ref, color='red', linewidth=1)
plt.show()

We can say that we have a good-fitting estimation, but do not have a great predictor.

### Multiple regression models

In [ ]:
y = data['university_rank']
X = data[['emoji_n', 'tweet_length', 'tweet_unique_length', 'positive']]
X = sm.add_constant(X)

In [ ]:
ks = sm.OLS(y, X)
ks_res =ks.fit()
ks_res.summary()

In [ ]:
y = data['university_rank']
X = data[['emoji_n', 'tweet_length', 'tweet_unique_length', 'positive', 'followers_count',
          'following_count', 'age_group', 'female', 'avr_w_length']]
X = sm.add_constant(X)

In [ ]:
ks = sm.OLS(y, X)
ks_res =ks.fit()
ks_res.summary()

In [ ]:
import statsmodels.formula.api as smf
ksf =  smf.ols('university_rank ~ emoji_n + tweet_length + tweet_unique_length + positive + followers_count + following_count + age_group + female + avr_w_length', data=data)
ksf_res = ksf.fit()
ksf_res.summary()

### Checking for colinearity

In [ ]:
import seaborn as sns
sns.pairplot(X[['emoji_n', 'tweet_length', 'tweet_unique_length', 'followers_count',
          'following_count', 'age_group', 'avr_w_length']])

In [ ]:
round(data.corr(),2)

In [ ]:
X = X.drop(columns = ['tweet_length', 'followers_count'], inplace = False)

In [ ]:
mod1 = sm.OLS(y, X)
mod1_res = mod1.fit()
mod1_res.summary()

### Regression diagnostics again

In [ ]:
from scipy import stats
sns.distplot(mod1_res.resid, fit=stats.norm)

In [ ]:
sns.boxplot(mod1_res.resid, showmeans=True)

In [ ]:
fig = sm.qqplot(mod1_res.resid, line='s')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

Y_max = y.max()
Y_min = y.min()

ax = sns.scatterplot(mod1_res.fittedvalues, y)
ax.set(ylim=(Y_min, Y_max))
ax.set(xlim=(Y_min, Y_max))
ax.set_xlabel("Predicted value of university rank")
ax.set_ylabel("Observed value of university rank")

X_ref = Y_ref = np.linspace(Y_min, Y_max, 100)
plt.plot(X_ref, Y_ref, color='red', linewidth=1)
plt.show()

### Standardize variables

standardized_a = ( a - a.mean() ) / a.std()

In [ ]:
from scipy import stats
y_norm = pd.Series(stats.zscore(y), name=y.name)
y_norm

In [ ]:
X_norm = X.loc[:, X.columns != "const"]
X_norm = pd.DataFrame(stats.zscore(X_norm))
X_norm = sm.add_constant(X_norm)
X_norm.columns = X.columns
check = pd.concat([round(X_norm.mean(axis=0)), round(X_norm.std(axis=0))], axis=1)
check.columns=["mean", "std dev"]
check

In [ ]:
modstd = sm.OLS(y_norm, X_norm)
modstd_res = modstd.fit()
modstd_res.summary()

In [ ]:
coeff = mod1_res.params
#coeff = coeff.iloc[(coeff.abs()*-1.0).argsort()] in order to sort coefficients
sns.barplot(x=coeff.values, y=coeff.index, orient='h')

In [ ]:
coeff = modstd_res.params
sns.barplot(x=coeff.values, y=coeff.index, orient='h')